<a href="https://colab.research.google.com/github/matheus896/crewai-examples/blob/main/Assistente_de_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crewai
!pip install litellm

In [22]:
import pandas as pd
import os
from crewai import Agent, Task, Crew, LLM
from litellm import completion

def main():
    """
    Função principal para inicializar e executar o Assistente de Aprendizado de Máquina do CrewAI.

    Esta função configura um assistente de aprendizado de máquina usando o modelo Llama 3 com a API ChatGroq.
    Ele fornece uma interface baseada em texto para os usuários definirem, avaliarem e resolverem problemas de aprendizado de máquina,
    interagindo com múltiplos agentes de IA especializados. A função exibe os resultados no console
    e os grava em um arquivo markdown.

    Passos:
    1. Inicializa a API ChatGroq com o modelo especificado e a chave da API.
    2. Exibe um texto introdutório sobre o Assistente de Aprendizado de Máquina CrewAI.
    3. Cria e configura quatro agentes de IA:
        - Agente_Definicao_Problema: Esclarece o problema de aprendizado de máquina.
        - Agente_Avaliacao_Dados: Avalia a qualidade e a adequação dos dados fornecidos.
        - Agente_Rec_Models: Recomenda modelos adequados de aprendizado de máquina.
        - Agente_Gerador_Codigo: Gera um código inicial em Python para o projeto.
    4. Solicita ao usuário que descreva seu problema de aprendizado de máquina.
    5. Verifica se há um arquivo .csv disponível no diretório atual e tenta lê-lo como um DataFrame.
    6. Define tarefas para os agentes com base na entrada do usuário e na disponibilidade dos dados.
    7. Cria uma instância do Crew com os agentes e tarefas e executa as tarefas.
    8. Imprime os resultados e os grava em um arquivo de saída markdown.
    """

os.environ["GROQ_API_KEY"] = "SUA_API_KEY"

llm = LLM (
    model = 'groq/llama3-8b-8192',
    temperature=0.1,
    api_key="SUA_API_KEY"
    )

texto_introdutorio = """
    O Assistente de Aprendizado de Máquina CrewAI foi projetado para guiar usuários através
    do processo de definição, avaliação e resolução de problemas de aprendizado de máquina.
    Ele utiliza uma equipe de agentes de IA, cada um com um papel específico, para esclarecer o problema,
    avaliar os dados, recomendar modelos adequados e gerar um código inicial em Python.
    Seja você um cientista de dados experiente ou iniciante, esta aplicação fornece insights valiosos e
    um ponto de partida para seus projetos de aprendizado de máquina.
    """

print(texto_introdutorio)



    O Assistente de Aprendizado de Máquina CrewAI foi projetado para guiar usuários através 
    do processo de definição, avaliação e resolução de problemas de aprendizado de máquina. 
    Ele utiliza uma equipe de agentes de IA, cada um com um papel específico, para esclarecer o problema, 
    avaliar os dados, recomendar modelos adequados e gerar um código inicial em Python. 
    Seja você um cientista de dados experiente ou iniciante, esta aplicação fornece insights valiosos e 
    um ponto de partida para seus projetos de aprendizado de máquina.
    


# AGENTES

In [18]:
Agente_Definicao_Problema = Agent(
        role='Agente_Definicao_Problema',
        goal="""esclarecer o problema de aprendizado de máquina que o usuário deseja resolver,
            identificando o tipo de problema (ex.: classificação, regressão) e quaisquer requisitos específicos.""",
        backstory="""Você é um especialista em compreender e definir problemas de aprendizado de máquina.
            Seu objetivo é extrair uma declaração de problema clara e concisa da entrada do usuário,
            garantindo que o projeto comece com uma base sólida.""",
        verbose=True,
        allow_delegation=False,
        llm=llm,
    )

Agente_Avaliacao_Dados = Agent(
        role='Agente_Avaliacao_Dados',
        goal="""avaliar os dados fornecidos pelo usuário, verificando sua qualidade,
            adequação ao problema e sugerindo etapas de pré-processamento, se necessário.""",
        backstory="""Você é especialista em avaliação e pré-processamento de dados.
            Sua tarefa é orientar o usuário na preparação de seu conjunto de dados para o modelo de aprendizado de máquina,
            incluindo sugestões para limpeza e aumento dos dados.""",
        verbose=True,
        allow_delegation=False,
        llm=llm,
    )

Agente_Rec_Models = Agent(
        role='Agente_Rec_Models',
        goal="""sugerir os modelos de aprendizado de máquina mais adequados com base na definição do problema
            e na avaliação dos dados, fornecendo justificativas para cada recomendação.""",
        backstory="""Como especialista em algoritmos de aprendizado de máquina, você recomenda modelos que melhor se encaixam
            no problema e nos dados do usuário. Você fornece insights sobre por que certos modelos podem ser mais eficazes do que outros,
            considerando classificação vs regressão e aprendizado supervisionado vs não supervisionado.""",
        verbose=True,
        allow_delegation=False,
        llm=llm,
    )

Agente_Gerador_Codigo = Agent(
        role='Agente_Gerador_Codigo',
        goal="""gerar um código inicial em Python para o projeto, incluindo carregamento de dados,
            definição do modelo e um loop básico de treinamento, com base nas descobertas da definição do problema,
            avaliação dos dados e recomendação do modelo.""",
        backstory="""Você é um especialista em gerar modelos de código inicial que os usuários podem personalizar para seus projetos.
                    Seu objetivo é dar aos usuários um ponto de partida para seus esforços de codificação.""",
        verbose=True,
        allow_delegation=False,
        llm=llm,
    )

# Tasks

In [19]:
descricao_problema_usuario = input("Descreva seu problema de aprendizado de máquina: ")
dados_enviados = False

    # Verifica se há um arquivo .csv no diretório atual
if any(file.endswith(".csv") for file in os.listdir()):
        arquivo_amostra = [file for file in os.listdir() if file.endswith(".csv")][0]
        try:
            # Tenta ler o arquivo como um DataFrame
            df = pd.read_csv(arquivo_amostra).head(5)

            # Se bem sucedido, define 'dados_enviados' como True
            dados_enviados = True

            # Exibe o DataFrame na aplicação
            print("Dados carregados e lidos como DataFrame com sucesso:")
            print(df)
        except Exception as e:
            print(f"Erro ao ler o arquivo: {e}")

if descricao_problema_usuario:

        tarefa_definir_problema = Task(
            description="""Clarificar e definir o problema de aprendizado de máquina,
                incluindo identificar o tipo de problema e os requisitos específicos.

                Problema do usuário:
                {problema_ml}
                """.format(problema_ml=descricao_problema_usuario),
            agent=Agente_Definicao_Problema,
            expected_output="Uma definição clara e concisa do problema de aprendizado de máquina. resposta apenas em PT-BR."
        )

if dados_enviados:
            tarefa_avaliar_dados = Task(
                description="""Avaliar os dados fornecidos pelo usuário quanto à qualidade e adequação,
                sugerindo etapas de pré-processamento ou aumento, se necessário.

                Aqui estão os dados do usuário:
                {df}
                O nome do arquivo é {arquivo_carregado}
                """.format(df=df.head(), arquivo_carregado=arquivo_amostra),
                agent=Agente_Avaliacao_Dados,
                expected_output="Uma avaliação da qualidade e adequação dos dados, com sugestões para pré-processamento ou aumento, se necessário. resposta apenas em PT-BR."
            )
else:
            tarefa_avaliar_dados = Task(
                description="""O usuário não enviou dados específicos para este problema,
                mas considere um conjunto de dados hipotético que possa ser útil para o problema de aprendizado de máquina deles.
                """,
                agent=Agente_Avaliacao_Dados,
                expected_output="Um conjunto de dados hipotético que possa ser útil para o problema do usuário, juntamente com as etapas de pré-processamento necessárias. resposta apenas em PT-BR."
            )

tarefa_recomendar_modelo = Task(
            description="""Sugerir modelos de aprendizado de máquina adequados para o problema definido e dados avaliados,
                           fornecendo justificativa para cada sugestão.""",
            agent=Agente_Rec_Models,
            expected_output="Uma lista de modelos de aprendizado de máquina adequados para o problema definido e dados avaliados, juntamente com a justificativa para cada sugestão. resposta apenas em PT-BR."
        )

tarefa_gerar_codigo = Task(
            description="""Gerar um código inicial em Python personalizado para o projeto do usuário, usando as recomendações do agente de recomendação de modelos,
                incluindo snippets para importação de pacotes, manipulação de dados, definição de modelo e treinamento.
                """,
            agent=Agente_Gerador_Codigo,
            expected_output="Snippets de código em Python para importação de pacotes, manipulação de dados, definição de modelo e treinamento, personalizados para o projeto do usuário, além de um resumo do problema e recomendações de modelos. resposta apenas em PT-BR.",
            context = [tarefa_recomendar_modelo]
        )

Descreva seu problema de aprendizado de máquina: Quero melhorar o índice de satisfação dos clientes que entram em contato pelo chat  e reduzir o tempo de espera na fila,  pois percebo que esses fatores têm impactado negativamente nas avaliações dos clientes.
Erro ao ler o arquivo: 'utf-8' codec can't decode bytes in position 15-16: invalid continuation byte


# Crew

In [20]:
equipe = Crew(
            agents=[Agente_Definicao_Problema, Agente_Avaliacao_Dados, Agente_Rec_Models, Agente_Gerador_Codigo],
            tasks=[tarefa_definir_problema, tarefa_avaliar_dados, tarefa_recomendar_modelo, tarefa_gerar_codigo],
            verbose=True
        )
resultado = equipe.kickoff()
resultado_str = str(resultado)

with open('saida.md', "w") as file:
    print('\n\nOs resultados foram exportados para saida.md')
    file.write(resultado_str)


# Agent: Agente_Definicao_Problema
## Task: Clarificar e definir o problema de aprendizado de máquina,
                incluindo identificar o tipo de problema e os requisitos específicos.

                Problema do usuário:
                Quero melhorar o índice de satisfação dos clientes que entram em contato pelo chat  e reduzir o tempo de espera na fila,  pois percebo que esses fatores têm impactado negativamente nas avaliações dos clientes.
                


# Agent: Agente_Definicao_Problema
## Final Answer: 
O problema de aprendizado de máquina é: Desenvolver um modelo de aprendizado de máquina que prediga a satisfação dos clientes que entram em contato pelo chat e identifique os fatores que influenciam no tempo de espera na fila, com o objetivo de melhorar o índice de satisfação dos clientes e reduzir o tempo de espera na fila.

O tipo de problema é uma tarefa de regressão, pois o objetivo é prever um valor contínuo (índice de satisfação e tempo de espera) com base em variá

In [21]:
from IPython.display import Markdown
Markdown(resultado.raw)

```
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

# Load dataset
df = pd.read_csv('dataset.csv')

# Preprocess data
le = LabelEncoder()
df['dia_semana'] = le.fit_transform(df['dia_semana'])
df['tipo_problema'] = le.fit_transform(df['tipo_problema'])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.drop('satisfacao_cliente', axis=1), df['satisfacao_cliente'], test_size=0.2, random_state=42)

# Define and train the model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Make predictions
y_pred = rf.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')

# Use the model to make predictions on new data
new_data = pd.DataFrame({'dia_semana': [1, 2, 3], 'hora_chamada': [10, 11, 12], 'tipo_problema': [0, 1, 2]})
new_data['satisfacao_cliente'] = rf.predict(new_data)
print(new_data)
```

This code includes the following:

* Import necessary libraries
* Load the dataset
* Preprocess the data by encoding categorical variables
* Split the data into training and testing sets
* Define and train a random forest regressor model
* Make predictions on the testing set
* Evaluate the model using mean squared error
* Use the model to make predictions on new data

Note that this is just a starting point, and the user may need to modify the code to suit their specific needs.

In [ ]:
"""
input usado de exemplo: "Quero melhorar o índice de satisfação dos clientes que entram em contato pelo chat
e reduzir o tempo de espera na fila, pois percebo que esses fatores têm impactado negativamente nas avaliações dos clientes."
"""